1/14 최초 구현 by 소연 

수정 및 테스트 시 본 파일이 아닌 사본 사용을 부탁드립니다.


1/18  수정 by 지현

* 회의록 음성 파일 일정 크기로 자르는 loop추가 --> 회의 전체 내용이 chked에 들어가도록 하였음
* 음성 파일 중 일부(2개) .UnknownValueError 발생하여 제거 --> 제거한 채로 갈지, 따로 처리해줘야 할지?

1/18 수정 by 소연

* UnknownValueError 발생하는 이슈 해결
* STT loop 함수화 --> 나중에 다른 것도 잘되는거 확인하면 spellchecker랑 class화 합시당

In [4]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Shareddrives/KPMG_Ideation
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pprint import pprint
from krwordrank.word import KRWordRank
from copy import deepcopy
import kss
import itertools
import unicodedata
import requests
from functools import reduce
from bs4 import BeautifulSoup
import string
import torch
from textrankr import TextRank
from lexrankr import LexRank
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
from pydub import AudioSegment
from konlpy.tag import Okt
import re
import nltk
# nltk.download('punkt')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives/KPMG_Ideation


In [5]:
# import pre-trained model -- frameBERT (pytorch GPU 환경 필요)
%cd /content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT
!pip install transformers 
import frame_parser
path="/content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT"
parser = frame_parser.FrameParser(model_path=path, language='ko')

/content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT
srl model: framenet
language: ko
version: 1.2
using viterbi: False
using masking: True
pretrained BERT: bert-base-multilingual-cased
using TGT special token: True
used dictionary:
	 /content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/kfn1.2_lu2idx.json
	 /content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/kfn1.2_lufrmap.json
	 /content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT/src/../koreanframenet/resource/info/mul_bio_frargmap.json
...loaded model path: /content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT
/content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT
...model is loaded


In [ ]:
##### below are permanently installed packages #####
# nb_path = '/content/notebooks'
# os.symlink('/content/drive/Shareddrives/KPMG_Ideation', nb_path)
# sys.path.insert(0, nb_path)
# !pip install --target=$nb_path pydub
# !pip install --target=$nb_path kss
# %cd /content/drive/Shareddrives/KPMG_Ideation/hanspell
# !python setup.py install
# !pip install --target=$nb_path transformers
# !apt-get update
# !apt-get g++ openjdk-8-jdk 
# !pip3 install --target=$nb_path konlpy
# !pip install --target=$nb_path soykeyword
# !pip install --target=$nb_path krwordrank
# !pip install --target=$nb_path bert
# !pip install --target=$nb_path textrankr
# !pip install --target=$nb_path lexrankr

In [1]:
# Due to google api credentials, SpeechRecognition needs to be installed everytime
!pip install SpeechRecognition
import speech_recognition as sr
#!pip install google-api-core==1.22.2
# !pip install --upgrade google-api-core
# !pip install --upgrade google-cloud-speech

In [6]:
files_path = ''
file_name = ''
def STT_loop(file_name, startMin = 0, startSec = 0, endMin = 4, endSec = 30):
    if file_name.split('.')[1] == 'mp3':    
        sound = AudioSegment.from_mp3(file_name)
        wav_filename = file_name.split('.')[0] + '.wav'
        file_handle = sound.export(file_name, format="wav")
        song = AudioSegment.from_wav(wav_filename)
    if file_name.split('.')[1] == 'm4a':
        track = AudioSegment.from_file(file_name,'m4a')
        wav_filename = file_name.replace('m4a', 'wav')
        file_handle = track.export(wav_filename, format='wav') 
        song = AudioSegment.from_wav(wav_filename)
    
    # Time to miliseconds
    startTime = startMin*60*1000+startSec*1000
    endTime = endMin*60*1000+endSec*1000

    num_frac=int(len(song)/endTime)
    for i in range(num_frac+1):
      extract = song[startTime:min(endTime,len(song))]
      extract.export('./audio_frac/frac_'+str(i)+'.wav', format="wav")
      startTime=endTime
      endTime+=endMin*60*1000+endSec*1000

    # Speech Recognition Loop
    txt=''
    for i in range(num_frac+1):
      AUDIO_FILE = os.path.join(os.path.dirname(os.path.abspath('data')), "audio_frac/frac_"+str(i)+".wav")

      # use the audio file as the audio source
      r = sr.Recognizer()
      with sr.AudioFile(AUDIO_FILE) as source:
          audio = r.record(source)  # read the entire audio file

      # recognize speech using Google Speech Recognition
      try:
          # for testing purposes, we're just using the default API key
          # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
          # instead of `r.recognize_google(audio)`
          txt += ' '+ r.recognize_google(audio, language='ko')
          print(str(i)+"/"+str(num_frac+1)+" done")
          #print("Google Speech Recognition:" + txt)
      except sr.UnknownValueError:
          print("Google Speech Recognition could not understand audio")
      except sr.RequestError as e:
          print("Could not request results from Google Speech Recognition service; {0}".format(e))
      
    return txt

In [7]:
%cd /content/drive/Shareddrives/KPMG_Ideation/data
file_name='audio_only_1.m4a'
txt = STT_loop(file_name)

/content/drive/Shareddrives/KPMG_Ideation/data
0/18 done
1/18 done
2/18 done
3/18 done
4/18 done
5/18 done
6/18 done
7/18 done
8/18 done
9/18 done
10/18 done
11/18 done
12/18 done
13/18 done
14/18 done
15/18 done
16/18 done
17/18 done


In [8]:
%cd /content/drive/Shareddrives/KPMG_Ideation/hanspell
from hanspell import spell_checker
chked=""
line = kss.split_sentences(txt)
for i in range(len(line)):
  if i%20==0:
    print(str(i)+"/"+str(len(line))+" lines done")
  line[i] = spell_checker.check(line[i])[2]
  #print("Checked spelling ",line[i])
  chked += "".join(line[i])
  chked += ". "

/content/drive/Shareddrives/KPMG_Ideation/hanspell
0/290 lines done
20/290 lines done
40/290 lines done
60/290 lines done
80/290 lines done
100/290 lines done
120/290 lines done
140/290 lines done
160/290 lines done
180/290 lines done
200/290 lines done
220/290 lines done
240/290 lines done
260/290 lines done
280/290 lines done


In [9]:
chked[:1000]

'문제가 있었다. 그 알고리즘으로 막상 해 놨길래 했는데 거기서 그 의료인들이 거의 다 빠지고 1순위에서 의료인들이 제일 필요한데 개를 다 빠지고 다른데 접종을 하는 것 때문에 막 난리가 났다고 그래가지고 알고리즘을 다시 만나기로 했어 나 뭐 이런 얘기 하다. 그래서 거기서 어떻게 그 어떤 자료들을 사용을 한다면은 아까 말씀했듯이 그 바이러스 노출도 직업에 따른 바이러스 노출도 그러나 이 그걸 가진 크게 두 개로 받고 멜론에서 나이가 65세 이상 해결하는 거 아닌가 25세 이하라던가 하면은 그 조금 더 가산점을 주는 거라던가 아니면은 뭐 직업 특성상 뭐 그 의도가 이런데 종사하면 가산점을 주는 거고 근데 반면에 이제 조금 문제가 어떻게 될 수 있냐면 직업 특성 만 가지고 하면 내 막상 재택근무자 아들이라도 산업 군이 막 그런 그쪽에서 켜면은 태권도는 아무런 위험에 노출되어 있지 않은데 그런 식으로 바다 쪽으로 받아가지고 오히려 더 그런 그런 문제들이 아직 해결할 때 많이 남았다고 찾아왔어요. 굉장히 좋은 지적이십니다. 그렇다면 직업 특성상 재택근무가 아예 불가능한 집중된 섞여 있을 거 같은데 직업 특성뿐만이 아니라 그냥 바이러스 노출도 제일 중요한 거 같기도 하고 그 그게 이제 목적에 따라서 누굴 먼저 맞춰야 되니까 조금 다른게 그 감염률을 나 죽고 싶으면 네 사실 뭐랄까 제일 활발하게 활동하는 새끼들을 먼저 맞춰야지 그래야지 지금 뭐 그 퍼트리고 다니는 애들이 안 걸려 있으니까 그래서 그런 경우 조금 야외활동이라던가 이런 것들이 활동이 많은 젊은 층들이 맞춰야 되는 반면에 치사율을 낮추고자를 목표로 하면은 우선은 노인 인구가 가장 큰 높으니까 애들 먼저 맞춰야 된다 이런 일이 있어가지고 달달 말씀이시죠 목적에 따라 달라진다. 목적에 따라 준다. 그리고 또 다른 이슈가 될 만한 게 또 배탈 개 중에 하나가 박신 백신을 맞고 나서 이제 어느 정도로 확진자 수가 누적돼서 증가하는 야 그걸 또 어느 도착할 것으로 보는 경우도 많잖아요. 그러려면 그렇죠 그런데 그럴 

In [10]:
chked[-1000:]

'연령 지역 그리고 아까 그 무슨 과목 얘기도 잠깐 나가서 어떤 거 같은데 그것처럼 약간 그런 변수 정말 넣어야 하는 변수들 이런 거 같아요. 그러면 각각 프로토타입을 작성을 하고 나중에 같이 이렇게 검술을 해서 제출하는 걸로 하는 거죠. 이게 친정을 아니거든 뭐 하고 저희가 나중에 다 같이 이거 결국은 짬뽕하면 되는 거고 하니까 저의 진짜 제안서는 같은 경우에도 저희가 사실 많이 섭취를 해 보겠음 이렇게 정리를 해 보자 정말 제안서를 제출한다는 식으로 저희가 정리를 갔다. 하는 시간을 가졌으면 합니다. 그게 또 의심이 있을 거 같아서 저희 생각 정리하는데 그러면은 그 언제 그 팀들이 알아서 그 뭐냐 그 이거를 완성에 와라 그 데이터를 정하면 될 거 같은데 아 그리고 까먹었는데 그리고도 돼 아까 데이터 얘기도 많이 나왔잖아요. 그래서 내가 이 모델을 했을 때 우리가 수집해야 할 데이터 구체적인 거 이런 데이터가 필요할 것 같다. 그리고 아까 그 백신 같은 경우에는 사실 저희가 정보를 의학지식이 정말 전문 지식이 있어야 되잖아요. 그런 지식을 우리가 뭘 어떻게 알아야 될 거 같다. 어떤 논문에 계속 참고해야 될 거 같고 어떤 쪽으로 우리가 지식을 키워야 될 거 같다. 이런 것도 포함해 주시면 괜찮을 거 그리고 버리게 6분 안에 승현 님 가시기 전에 날짜를 바꿔 보면은 표를 했을 때 저번에 아 금요일 토요일이 정보가 다 되고 그러면 7시에는 괜찮으신 건가요. 조금씩 뭐 좀 나는 좀 더 앞으로 당겨서 괜찮다 뭐 이런 건 있으세요. 저녁 시간이 애매해서 그냥 7시에는 괜찮으시면 7시 좋아요. 그러면 화요일은 통 정말 되는 사람이 별로 없고 수요일 금요일에 뭔가 너무 먼 기분이 나서 지금 제가 좀 꺼려지네요. 지금 수요일 같은 경우 안 된다고 하신 분이 가영이가 형님 아예 안 된대요 7시 수요일에 생일입니다. 목요일 14일네 알겠습니다. 괜찮아요. 그냥 목요일 알겠습니다. 그럼 목요일 7시까지 거리가 어떻게 돼 한서를 자기 방식으로 한번 정리를 해서 오는 걸로 합시다.

In [11]:
np.array(['w'])
%cd /content/drive/Shareddrives/KPMG_Ideation/data
t=open('./texts/'+file_name.split('.')[0] + '.txt', "w")
n=t.write(chked)
t.close()

/content/drive/Shareddrives/KPMG_Ideation/data


In [15]:
okt = Okt()
class Text(): 
    def __init__(self, text):
        text = re.sub("'", ' ', text)
        paragraphs = text.split('\n')
        self.text = text
        self.paragraphs = [i for i in paragraphs if i]
        self.counts = len(self.paragraphs)
        self.docs = [kss.split_sentences(paragraph) for paragraph in paragraphs if kss.split_sentences(paragraph)]
        self.newtext = deepcopy(self.text)
        print("TEXT")

    def findall(self, p, s):
        i = s.find(p)
        while i != -1:
            yield i
            i = s.find(p, i + 1)
      
    def countMatcher(self, sentences, paragraph_no):
        paragraph = self.docs[paragraph_no]
        total_no = len(paragraph)
        vec = [0] * total_no
        
        for idx, candidate in enumerate(paragraph):
            for sentence in sentences:
                if sentence[:4] in candidate:
                    vec[idx] += 1
        return vec


class Highlight(Text):
    def __init__(self, text):
        super().__init__(text)
        print("Highlight")
        wordrank_extractor = KRWordRank(min_count=3, max_length=10)
        self.keywords, rank, graph = wordrank_extractor.extract(self.paragraphs)
        self.path = "/content/drive/Shareddrives/KPMG_Ideation/OpenInformationExtraction/frameBERT"
        p = []
        kw = []
        for k, v in self.keywords.items():
            p.append(okt.pos(k))
            kw.append(k)
        words = self.text.split(' ')
        s = set()
        keylist = [word for i in kw for word in words if i in word]
        keylist = [i for i in keylist if len(i)>2]
        for i in keylist:
            if len(i)>2:
              s.add(i)
        # print("KEYLIST: ",keylist)

        p = [okt.pos(word) for word in s]
        self.s = set()
        for idx in range(len(p)):
            ls = p[idx]
            for tags in ls:
              word,tag = tags
              if tag == "Noun":
                if len(word)>=2:
                  self.s.add(word)
        self.keys = []
        for temp in self.s:
            self.keys.append(" " + str(temp))
        print("KEYWORDS: ", self.keys)

    # def add_tags_conj(self, txt):
    #     conj = '그리고, 그런데, 그러나, 그래도, 그래서, 또는, 및, 즉, 게다가, 따라서, 때문에, 아니면, 왜냐하면, 단, 오히려, 비록, 예를 들어, 반면에, 하지만, 그렇다면, 바로, 이에 대해'
    #     conj = conj.replace("'", "")
    #     self.candidates = conj.split(",")
    #     self.newtext = deepcopy(txt)
    #     self.idx = [(i, i + len(candidate)) for candidate in self.candidates for i in
    #                     self.findall(candidate, txt)]
    #     for i in range(len(self.idx)):
    #         try:
    #             self.idx = [(start, start + len(candidate)) for candidate in self.candidates for start in
    #                         self.findall(candidate, self.newtext)]
    #             word = self.newtext[self.idx[i][0]:self.idx[i][1]]
    #             self.newtext = word.join([self.newtext[:self.idx[i][0]], self.newtext[self.idx[i][1]:]])
    #         except:
    #             pass
    #     return self.newtext

In [13]:
class Summarize(Highlight):
    def __init__(self, text, paragraph_no):
      super().__init__(text)
      print("length of paragraphs ",len(self.paragraphs))
      self.txt = self.paragraphs[paragraph_no]
      self.paragraph_no = paragraph_no

    def summarize(self):
        url = "https://api.smrzr.io/v1/summarize?num_sentences=5&algorithm=kmeans"
        headers = {
            'content-type': 'raw/text',
            'origin': 'https://smrzr.io',
            'referer': 'https://smrzr.io/',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'same-site',
            "user-agent": "Mozilla/5.0"
        }
        resp = requests.post(url, headers=headers, data= self.txt.encode('utf-8'))
        assert resp.status_code == 200
        summary = resp.json()['summary']
        temp = summary.split('\n')
        print("BERT: ", temp)
        return temp


    def summarizeTextRank(self):
        tr = TextRank(sent_tokenize)
        summary = tr.summarize(self.txt, num_sentences=5).split('\n')
        print("Textrank: ",summary)
        return summary


    def summarizeLexRank(self):
        lr = LexRank()
        lr.summarize(self.txt)
        summaries = lr.probe()
        print("Lexrank: ",summaries)
        return summaries
LexRank().summarize()
    def ensembleSummarize(self):
        a = np.array(self.countMatcher(self.summarize(), self.paragraph_no))
        
        try:
          b = np.array(self.countMatcher(self.summarizeLexRank(), self.paragraph_no))
        except:
          b = np.zeros_like(a)
        c = np.array(self.countMatcher(self.summarizeTextRank(),self.paragraph_no))
        result= a+b+c
        i, = np.where(result == max(result))
        txt, index = self.docs[self.paragraph_no][i[0]], i[0]
        return txt, index

In [16]:
result = chked
high = Highlight(result)

TEXT
Highlight
KEYWORDS:  [' 직업', ' 여기', ' 거나', ' 확인', ' 짬뽕', ' 시기', ' 해결', ' 영어', ' 경제활동', ' 각자', ' 인구', ' 투여', ' 변수', ' 엄마', ' 프로토타입', ' 무시', ' 플러스', ' 주시', ' 분야', ' 개발', ' 반복', ' 이중', ' 잠시', ' 다른', ' 공모전', ' 회사', ' 건가', ' 혹시', ' 직접', ' 정해진', ' 사실', ' 감염', ' 기술', ' 파악', ' 순위', ' 언어', ' 이해', ' 템플릿', ' 프로젝트', ' 법률용어', ' 각각', ' 달라', ' 가능', ' 저희', ' 수도권', ' 웨이트', ' 훈련', ' 이상', ' 도도', ' 회로', ' 구글', ' 음성', ' 대신', ' 인식', ' 처리', ' 공급망', ' 설정', ' 머리', ' 코로나', ' 운송', ' 고양이', ' 자동', ' 사망률', ' 질병', ' 기관', ' 방안', ' 한국어', ' 전환', ' 제안', ' 스피츠', ' 일정', ' 문제', ' 바이러스', ' 타이틀', ' 목표', ' 이전', ' 디자인', ' 반면', ' 보시', ' 자체', ' 전화', ' 산정', ' 처음', ' 체제', ' 요약', ' 화로', ' 서울대', ' 진행', ' 접종', ' 자원', ' 서울시', ' 목적', ' 보이스', ' 유동', ' 활동', ' 옛날', ' 메인', ' 교체', ' 기반', ' 주제', ' 감염병', ' 승현', ' 서울', ' 위로', ' 언제', ' 지역', ' 형님', ' 가지', ' 이민호', ' 방식', ' 문서', ' 특성', ' 학습', ' 이면', ' 얼마나', ' 조금', ' 의견', ' 얘기', ' 프로세스', ' 근거', ' 목요일', ' 면역', ' 말씀', ' 유리', ' 공급', ' 시작', ' 노출', ' 순서대로', ' 약간', ' 최적화', ' 순서', ' 따로따로', ' 작성', ' 시

In [ ]:
summarizer = Summarize(chked, 0)
sum, id = summarizer.ensembleSummarize()
print("summarized ",sum)

In [ ]:
sum

'그렇다면 직업 특성상 재택근무가 아예 불가능한 집중된 섞여 있을 거 같은데 직업 특성뿐만이 아니라 그냥 바이러스 노출도 제일 중요한 거 같기도 하고 그 그게 이제 목적에 따라서 누굴 먼저 맞춰야 되니까 조금 다른게 그 감염률을 나 죽고 싶으면 네 사실 뭐랄까 제일 활발하게 활동하는 새끼들을 먼저 맞춰야지 그래야지 지금 뭐 그 퍼트리고 다니는 애들이 안 걸려 있으니까 그래서 그런 경우 조금 야외활동이라던가 이런 것들이 활동이 많은 젊은 층들이 맞춰야 되는 반면에 치사율을 낮추고자를 목표로 하면은 우선은 노인 인구가 가장 큰 높으니까 애들 먼저 맞춰야 된다 이런 일이 있어가지고 달달 말씀이시죠 목적에 따라 달라진다.'

- 사용자 인식(speaker identification)이 됐으면 좋겠다 -- clova NOTE 사용시 해결
>무료 api는 supervised만 있는 듯

- 시간단위로 잘리는 것 루프 만들기

- 기본 웹프레임워크 만들기

- 아웃풋 어떤 모양일지?
